## Generacion de la clase_ternaria

In [ ]:
require( "data.table" )

# leo el dataset
dataset <- fread("/content/datasets/competencia_01_crudo.csv" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
    "pos" = .I,
    numero_de_cliente,
    periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 ) ]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]


# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

fwrite( dataset,
    file =  "/content/datasets/competencia_01.csv.gz",
    sep = ","
)

Loading required package: data.table



In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
202101,BAJA+1,622
202101,BAJA+2,825
202101,CONTINUA,160080
202102,BAJA+1,831
202102,BAJA+2,1032
202102,CONTINUA,160292
202103,BAJA+1,1039
202103,BAJA+2,951
202103,CONTINUA,161119


### 2.2 Optimizacion Hiperparámetros

### 2.2.1 Inicio

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Oct 11 11:43:00 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,726089,38.8,1454651,77.7,1454651,77.7
Vcells,1423194,10.9,169514316,1293.3,211568341,1614.2


### 2.2.2 Carga de Librerias

Esta parte lleva varios minutos la primera vez en Google Colab

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: parallel

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings


Loading required package: primes

Loading required package: rlist

Loading required packa

### 2.2.3 Definicion de Parametros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

In [ ]:
PARAM <- list()
PARAM$experimento <- 4923 # también 4921
PARAM$semilla_primigenia <- 99991 # uso distinta semilla para cada experimento


In [ ]:
# training y future
#entreno 202102,202103,202103
PARAM$train <- c(202101, 202102, 202103)
PARAM$train_final <- c(202101, 202102, 202103)
PARAM$future <- c(202104)
PARAM$semilla_kaggle <- 314159

In [ ]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM$trainingstrategy$undersampling <- 0.1

In [ ]:
# Parametros LightGBM

PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)


Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization
<br> si es un numero entero debe ir  makeIntegerParam
<br> si es un numero real (con decimales) debe ir  makeNumericParam
<br> es muy importante leer cuales son un lower y upper  permitidos y ademas razonables

In [ ]:
# Aqui se cargan los bordes de los hiperparametros de la BO
PARAM$hypeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_iterations", lower = 8L, upper = 2048L),
  makeNumericParam("learning_rate", lower = 0.005, upper = 0.06),
  makeNumericParam("feature_fraction", lower = 0.45, upper = 0.8),
  makeIntegerParam("num_leaves", lower = 8L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 10L, upper = 3500L),
  makeNumericParam("lambda_l1", lower = 0.0, upper = 5.0),
  makeNumericParam("lambda_l2", lower = 0.01, upper = 10.0),
  makeNumericParam("bagging_fraction", lower = 0.5, upper = 1.0),
  makeIntegerParam("bagging_freq", lower = 0L, upper = 5L),
  makeNumericParam("min_gain_to_split", lower = 0.0, upper = 10.0),
  makeNumericParam("min_sum_hessian_in_leaf", lower = 0.001, upper = 8.0),
  makeIntegerParam("max_depth", lower = -1L, upper = 16L),
  makeIntegerParam("max_bin", lower = 63L, upper = 511L)
)


A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
<br> 30 es un valor muy tacaño, pero corre rápido
<br> deberia partir de 50, alcanzando los 100 si se dispone de tiempo

In [ ]:
PARAM$hyperparametertuning$iteraciones <- 70 # iteraciones bayesianas

In [ ]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

In [ ]:
# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )

  return( drealidad )
}

In [ ]:
# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}

### 2.2.4  Preprocesamiento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/competencia_01.csv.gz", stringsAsFactors= TRUE)

PRIMERO CORRIJO EL DATA DRIFTING

Corrijo variables en pesos por el valor del dólar

In [ ]:
library(data.table)

# Definir los Factores de Conversión (Tipo de Dólar de Compra y Venta)
FACTORES_DOLAR <- data.table(
    foto_mes = c(202101, 202102, 202103, 202104, 202105, 202106),
    dolar_compra = c(84.51, 87.62, 90.20, 91.93, 93.23, 94.09),
    dolar_venta = c(90.79, 93.73, 96.31, 98.21, 99.50, 100.37)
)

# Clasificar las Variables por Naturaleza Financiera

# A. Ingresos, Saldos, Límites, Inversiones, Rentabilidad (ACTIVO / PODER DE COMPRA)
# USAR Dólar de COMPRA (menor valor)
vars_compra <- c(
    "mrentabilidad", "mrentabilidad_annual", "mactivos_margen", "mpasivos_margen",
    "mcuenta_corriente_adicional", "mcuenta_corriente", "mcaja_ahorro",
    "mcaja_ahorro_adicional", "mcuentas_saldo", "mpayroll", "mpayroll2",
    "mprestamos_personales", "mprestamos_prendarios", "mprestamos_hipotecarios",
    "mplazo_fijo_pesos", "minversion1_pesos", "minversion2",
    "mtransferencias_recibidas", "mextraccion_autoservicio", "mcheques_depositados",
    "Master_mfinanciacion_limite", "Master_msaldototal", "Master_msaldopesos",
    "Master_mlimitecompra", "Master_madelantopesos", "Master_mpagado",
    "Master_mpagospesos", "Master_mpagominimo",
    "Visa_mfinanciacion_limite", "Visa_msaldototal", "Visa_msaldopesos",
    "Visa_mlimitecompra", "Visa_madelantopesos", "Visa_mpagado",
    "Visa_mpagospesos", "Visa_mpagominimo"
)

# B. Consumo, Gastos, Costos, Comisiones, Débitos, Cheques Emitidos (GASTO / COSTO)
# USAR Dólar de VENTA (mayor valor)
vars_venta <- c(
    "mcomisiones", "mcomisiones_mantenimiento", "mcomisiones_otras", "mforex_buy",
    "mforex_sell", "mtransferencias_emitidas", "mcheques_emitidos",
    "mcheques_depositados_rechazados", "mcheques_emitidos_rechazados", "matm",
    "matm_other", "mtarjeta_visa_consumo", "mtarjeta_master_consumo",
    "mcuenta_debitos_automaticos", "mpagodeservicios", "mpagomiscuentas",
    "mcajeros_propios_descuentos", "mtarjeta_visa_descuentos", "mtarjeta_master_descuentos",
    "Master_mconsumospesos", "Master_mconsumototal",
    "Visa_mconsumospesos", "Visa_mconsumototal"
)

# Mergear el tipo de cambio al dataset
# Esto adjunta las columnas 'dolar_compra' y 'dolar_venta' según el mes
dataset <- merge(dataset, FACTORES_DOLAR, by = "foto_mes", all.x = TRUE)

# Función de Dolarización (Aplicación In-Place)
dolarizar_vars <- function(dt, vars_list, tipo_cambio_col) {
    # Evitar divisiones por cero (aunque poco probable con tasas reales)
    dt[, tasa := get(tipo_cambio_col) + 1e-9]

    for (v in vars_list) {
        # Asegurar que la columna es numérica (necesario si hay NA o tipos mixtos)
        dt[, (v) := as.numeric(get(v))]

        # Reemplazo in-place: valor_en_pesos / tasa_de_cambio
        dt[, (v) := get(v) / tasa]
    }
    dt[, tasa := NULL] # Limpiar columna temporal
}

# Aplicar la conversión

# Aplicar Dólar de Compra
dolarizar_vars(dataset, vars_compra, "dolar_compra")

# Aplicar Dólar de Venta
dolarizar_vars(dataset, vars_venta, "dolar_venta")

# Limpieza final: Eliminar columnas de tipo de cambio
dataset[, `:=`(dolar_compra = NULL, dolar_venta = NULL)]

cat("Conversión a dólares completada para", length(vars_compra) + length(vars_venta), "variables monetarias.\n")

Conversión a dólares completada para 59 variables monetarias.


Corrijo el efecto del aguinaldo en 202106 por un factor, en algunas variables

In [ ]:
library(data.table)

# --- Configuración ---
vars_aguinaldo_individual <- c(
    "mcuentas_saldo",
    "mpayroll",
    "mpayroll2",
    "mcaja_ahorro",
    "mcuenta_corriente",
    "mtarjeta_visa_consumo",
    "Visa_mconsumospesos"
)
UMBRAL_AUMENTO <- 1.5      # 50% de aumento (Valor Actual / Media Histórica > 1.5)
FACTOR_CORRECCION <- 1.5   # Factor por el que dividir el valor de 202106 si excede el umbral

#  Asegurar ordenamiento
setorder(dataset, numero_de_cliente, foto_mes)

# Calcular la media histórica (Acumulada) hasta el mes anterior para cada cliente
# Usaremos una media móvil/acumulada (f_prom_cum) para obtener una base robusta.

f_prom_cum <- function(x) {
  # Calcula el promedio acumulado, excluyendo el mes actual (histórico)
  acum_sum <- cumsum(replace(x, is.na(x), 0))
  acum_count <- seq_along(x)

  # Para el mes 'i', la media histórica es: Suma(1 a i-1) / Count(1 a i-1)
  # Creamos una columna temporal de media histórica (rolling mean)
  prom_historico <- shift(acum_sum, 1, NA, "lag") / shift(acum_count, 1, NA, "lag")

  # Manejo de NA para los primeros meses (donde no hay historial)
  prom_historico[is.na(prom_historico)] <- 0 # O se puede dejar en NA, pero 0 es más seguro para la división

  return(prom_historico)
}

# Aplicar el cálculo de la media histórica (temporal) para la detección
for (v in vars_aguinaldo_individual) {
    col_mean_hist <- paste0("mean_hist_", v)
    dataset[, (col_mean_hist) := f_prom_cum(get(v)), by = numero_de_cliente]
}

In [ ]:
# Inicializar el marcador de corrección para impresión
dataset[, fue_corregido := FALSE]

for (v in vars_aguinaldo_individual) {
    col_mean_hist <- paste0("mean_hist_", v)

    # Definir la condición de corrección

    # Condición: foto_mes es 202106 Y (Valor Actual / Media Histórica) > UMBRAL_AUMENTO
    condicion_correccion_v <- dataset$foto_mes == 202106 &
                              !is.na(dataset[[col_mean_hist]]) &
                              dataset[[col_mean_hist]] != 0 &
                              (dataset[[v]] / dataset[[col_mean_hist]]) > UMBRAL_AUMENTO

    # Aplicar la corrección IN-PLACE
    # IMPORTANTE: Reemplazamos la columna original (v) con el valor corregido.
    # Usamos as.numeric() para asegurar que el resultado de la división se almacene correctamente
    dataset[condicion_correccion_v,
            (v) := as.numeric(get(v) / FACTOR_CORRECCION)]

    # Marcar las filas que cumplen la condición
    dataset[condicion_correccion_v, fue_corregido := TRUE]
}

# Limpieza: Eliminar las medias históricas temporales
cols_medias_temp <- paste0("mean_hist_", vars_aguinaldo_individual)
dataset[, (cols_medias_temp) := NULL]

In [ ]:
# Imprimir los clientes y variables corregidas
cols_a_mostrar <- c("numero_de_cliente", "foto_mes", vars_aguinaldo_individual)

tabla_corregida_final <- dataset[foto_mes == 202106 & fue_corregido == TRUE,
                                 .SD,
                                 .SDcols = cols_a_mostrar]

 Limpieza final del indicador
dataset[, fue_corregido := NULL]


if (nrow(tabla_corregida_final) > 0) {
    cat(paste0("Se corrigieron ", nrow(tabla_corregida_final),
               " clientes en 202106 debido a una variación > 50% respecto a su media histórica (01-05).\n"))
    cat("\nPrimeros clientes corregidos con sus valores finales (corregidos):\n")
    print(head(tabla_corregida_final))

    # Nota: Los valores mostrados en esta tabla son los VALORES FINALES ya corregidos
    # (reemplazados in-place en el dataset).
} else {
    cat("No se encontró ningún cliente en 202106 cuya variación superara el 50% en las variables de aguinaldo respecto a su media histórica.\n")
}

Se corrigieron 115915 clientes en 202106 debido a una variación > 50% respecto a su media histórica (01-05).

Primeros clientes corregidos con sus valores finales (corregidos):
   numero_de_cliente foto_mes mcuentas_saldo mpayroll mpayroll2 mcaja_ahorro
               <int>    <int>          <num>    <num>     <num>        <num>
1:         249221323   202106       93.56882 1218.770         0     119.1303
2:         249234235   202106     -163.41262    0.000         0       0.0000
3:         249244449   202106     2583.62313    0.000         0    2140.8833
4:         249255456   202106    12748.12297 3148.429         0    3838.0639
5:         249255747   202106     1387.91115 3785.864         0    1367.0571
6:         249257429   202106      617.88947    0.000         0     717.2620
   mcuenta_corriente mtarjeta_visa_consumo Visa_mconsumospesos
               <num>                 <num>               <num>
1:         -7.380168              262.3166            106.9488
2:       -132.6774

INGENIERÍA DE ATRIBUTOS

In [ ]:
# Lista de variables de saldo líquido a sumar (asumiendo que ya están en USD)
vars_saldos_liquidos <- c(
    "mcuenta_corriente_adicional",
    "mcuenta_corriente",
    "mcaja_ahorro",
    "mcaja_ahorro_adicional"
)

# Creamos la nueva variable mCuentas_Total como la suma de los saldos líquidos
dataset[, mCuentas_Total := rowSums(.SD, na.rm = TRUE), .SDcols = vars_saldos_liquidos]

cat("Variable mCuentas_Total creada con la suma de los saldos líquidos.\n")

Variable mCuentas_Total creada con la suma de los saldos líquidos.


In [ ]:
library(data.table)

# Lista de variables que componen el patrimonio (todos en USD)
vars_patrimonio <- c(
    "mCuentas_Total",           # Ya incluye caja de ahorro y cuenta corriente dolarizadas
    "mplazo_fijo_dolares",      # Ya está en USD
    "mplazo_fijo_pesos",        # Dolarizada
    "minversion1_dolares",      # Ya está en USD
    "minversion1_pesos",        # Dolarizada
    "minversion2"               # Dolarizada (asumiendo que era en pesos)
)

# Creamos la nueva variable mPatrimonio_Total
# Usamos rowSums con na.rm=TRUE para que los NAs (clientes sin esa inversión) se traten como 0.
dataset[, mPatrimonio_Total := rowSums(.SD, na.rm = TRUE), .SDcols = vars_patrimonio]

cat("Variable mPatrimonio_Total creada exitosamente.\n")

Variable mPatrimonio_Total creada exitosamente.


Lags y Delta Lags

In [ ]:
library(data.table)

# Asegurar que el dataset sea un data.table
if (!is.data.table(dataset)) {
  dataset <- as.data.table(dataset)
}

## Preparación y Ordenamiento

# Columnas a excluir de la generación de lags
cols_excluir <- c("numero_de_cliente", "foto_mes", "clase_ternaria")

# Identificar las columnas a las que se les aplicará el lag
cols_lag <- setdiff(names(dataset), cols_excluir)

# Asegurar el orden temporal para que shift funcione correctamente dentro de cada cliente
setorder(dataset, numero_de_cliente, foto_mes)

## Cálculo de Lag_1 y Delta_lag_1 (Manteniendo tu código original)

# Aplico lag de 1 mes (lag_1)
dataset[, paste0("lag_1_", cols_lag) := lapply(.SD, shift, 1, NA, "lag"),
        by = numero_de_cliente, .SDcols = cols_lag]

# Cálculo de Delta_lag_1 (Actual - Lag_1)
for (v in cols_lag) {
  dataset[, paste0("Delta_lag_1_", v) := get(v) - get(paste0("lag_1_", v))]
}

## Cálculo de Lag_2 y Delta_lag_2 (Nuevas variables)

# Aplico lag de 2 meses (lag_2)
# Usamos n=2 en la función shift
dataset[, paste0("lag_2_", cols_lag) := lapply(.SD, shift, 2, NA, "lag"),
        by = numero_de_cliente, .SDcols = cols_lag]

# Cálculo de Delta_lag_2 (Actual - Lag_2)
# NOTA: Esto captura la diferencia del valor actual respecto a hace dos meses.
for (v in cols_lag) {
  dataset[, paste0("Delta_lag_2_", v) := get(v) - get(paste0("lag_2_", v))]
}

for (v in cols_lag) {
  dataset[, paste0("Delta_lag_1_lag_2_", v) := get(paste0("lag_1_", v)) - get(paste0("lag_2_", v))]
}

In [ ]:
nrow(dataset)
ncol(dataset)

[1] 978439

[1] 927

Pendiente

In [ ]:
# Función de Pendiente con Ventana Fija (W)
# W es la ventana de meses a considerar (ej. 3 meses)
f_pendiente_ventana <- function(x, W = 3) {
  # Función para calcular la pendiente de una serie temporal corta
  calc_pend <- function(ventana) {
    ventana <- ventana[!is.na(ventana)]
    if (length(ventana) < 2) return(NA_real_)
    t <- 1:length(ventana)
    # Usa 'coef' para extraer el coeficiente de la variable 't' (el tiempo)
    return(coef(lm(ventana ~ t))[2])
  }

  # Aplica una ventana móvil (rollapply)
  # En data.table, se simula con 'frollapply' si está disponible (versión > 1.13) o un loop

  # Usando frollapply de data.table (rápido para ventanas fijas)
  # Si tu versión de data.table lo soporta:
  if (exists("frollapply")) {
    return(frollapply(x, W, calc_pend, align = "right", fill = NA_real_))
  } else {
    # Alternativa manual para versiones antiguas
    pendientes <- rep(NA_real_, length(x))
    for (i in W:length(x)) {
      pendientes[i] <- calc_pend(x[(i - W + 1):i])
    }
    return(pendientes)
  }
}

# Aplicación:
pend_vars_ventana <- c("ctrx_quarter", "cproductos")
for (v in pend_vars_ventana) {
  newcol <- paste0("pend_3m_", v)
  # W=3 para los últimos 3 meses
  dataset[, (newcol) := f_pendiente_ventana(get(v), W = 3), by = numero_de_cliente]
}

In [ ]:
nrow(dataset)
ncol(dataset)

[1] 978439

[1] 929

Ranking, promedio y rangos (max-min)

In [ ]:
library(data.table)
library(stats)  # Para asegurar que ecdf esté disponible

# Funciones Acumuladas (Vectorizadas/Semi-Vectorizadas)

# Rango Acumulado (Vectorizado)
f_rango_cum <- function(x) {
  # Asegura que el NA no se propague inmediatamente si el primer valor es NA
  x_no_na <- replace(x, is.na(x), x[1]) # Reemplazo temporal para que cummin/max funcione
  cummax(x) - cummin(x)
}

# Promedio Acumulado (Vectorizado) - Correcto, maneja NAs reemplazándolos con 0 en la suma
f_prom_cum <- function(x) {
  cumsum(replace(x, is.na(x), 0)) / seq_along(x)
}


f_ranking <- function(x) {
  ranks <- rep(NA_real_, length(x))

  for (i in seq_along(x)) {
    val <- x[i]

    if (is.na(val)) next

    # Historial de valores ANTERIORES al mes actual
    historial <- if (i > 1) x[1:(i - 1)] else numeric(0)
    historial <- historial[!is.na(historial)] # Remover NAs del historial

    # Si no hay historial, se inicializa a 0.5 (neutral)
    if (length(historial) == 0) {
      ranks[i] <- 0.5
      next
    }

    # --- Lógica de Ranking con Cero Fijo ---
    if (val == 0) {
      ranks[i] <- 0 # El valor 0 debe ser el punto central/neutro (0)
    } else if (val > 0) {
      # 1. POSITIVOS (Rango [0, 1])
      historial_pos <- historial[historial > 0]
      if (length(historial_pos) > 0) {
        # El ECDF ya da un ranking de 0 a 1.
        ranks[i] <- ecdf(historial_pos)(val)
      } else {
        # Si no hay historial positivo, es el más alto (1)
        ranks[i] <- 1
      }
    } else { # val < 0
      # 2. NEGATIVOS (Rango [-1, 0])
      historial_neg <- historial[historial < 0]

      if (length(historial_neg) > 0) {
        # Para obtener un ranking de 0 a -1, aplicamos ECDF sobre
        # los valores NEGATIVOS de -historial (transformados a positivos)
        # y luego multiplicamos el resultado por -1.

        # El ECDF se aplica al valor -val (que es positivo) dentro de
        # la distribución de -historial_neg. Esto da 1 para los valores
        # más cercanos a cero (los menos negativos) y 0 para los más lejanos.
        # Multiplicar por -1 invierte el orden y el signo:
        # Rank 1 -> -1, Rank 0 -> 0.
        ranks[i] <- -ecdf(-historial_neg)(-val)

      } else {
        # Si no hay historial negativo, es el más bajo (-1)
        ranks[i] <- -1
      }
    }
  }
  return(ranks)
}


# Aplicación de Features al dataset COMPLETO


# RANGOS (vectorizado)
rango_vars <- c("mcuentas_saldo", "Master_msaldototal", "Visa_msaldototal")
for (v in rango_vars) {
  newcol <- paste0("rango_acum_", v)
  # Aplicación directa sobre el dataset completo
  dataset[, (newcol) := f_rango_cum(get(v)), by = numero_de_cliente]
}

# PROMEDIOS (vectorizado)
prom_vars <- c("mpasivos_margen", "cproductos",
               "Master_mlimitecompra", "Visa_mlimitecompra",
               "ctarjeta_visa", "ctarjeta_master", "ctrx_quarter")
for (v in prom_vars) {
  newcol <- paste0("prom_acum_", v)
  # Aplicación directa sobre el dataset completo
  dataset[, (newcol) := f_prom_cum(get(v)), by = numero_de_cliente]
}

# RANKINGS (requiere función incremental)
rank_vars <- c("mtarjeta_visa_consumo", "mtarjeta_master_consumo",
               "Master_mconsumototal", "Visa_mconsumototal",
               "Master_cconsumos", "Visa_cconsumos", "ctrx_quarter",
               "cpayroll_trx","mpayroll","cliente_edad")

dataset[, (rank_vars) := lapply(.SD, as.numeric), .SDcols = rank_vars]

for (v in rank_vars) {
  # newcol <- paste0("rank_acum_", v)
  # Aplicación directa sobre el dataset completo
  dataset[, (v) := f_ranking(get(v)), by = numero_de_cliente]
}

In [ ]:
nrow(dataset)
ncol(dataset)

[1] 978439

[1] 939

In [ ]:
dataset_train <- dataset[foto_mes %in% PARAM$train]

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [ ]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train[, azar := runif(nrow(dataset_train))]
dataset_train[, training := 0L]

dataset_train[
  foto_mes %in%  PARAM$train &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [ ]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train[training == 1L, clase01],
  free_raw_data= FALSE
)

nrow(dtrain)
ncol(dtrain)

[1] 53249

[1] 938

2.2.5 Configuracion Bayesian Optimization

In [ ]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la AUC en cross validation del modelo entrenado

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelocv <- lgb.cv(
    data= dtrain,
    nfold= PARAM$hyperparametertuning$xval_folds,
    stratified= TRUE,
    param= param_completo
  )

  # obtengo la ganancia
  AUC <- modelocv$best_score

  # hago espacio en la memoria
  rm(modelocv)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

In [ ]:
# Aqui comienza la configuracion de la Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_AUC_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hypeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


2.2.6 Corrida Bayesian Optimization

In [ ]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Sat Oct 11 12:19:59 2025 AUC 0.924349689933132

Sat Oct 11 12:21:24 2025 AUC 0.920671520388602

Sat Oct 11 12:23:50 2025 AUC 0.926772310162526

Sat Oct 11 12:27:01 2025 AUC 0.922573211779931

Sat Oct 11 12:28:29 2025 AUC 0.927740301908169

Sat Oct 11 12:31:01 2025 AUC 0.925818354950716

Sat Oct 11 12:32:00 2025 AUC 0.921110241165373

Sat Oct 11 12:32:28 2025 AUC 0.919331050457514

Sat Oct 11 12:34:20 2025 AUC 0.925737365664357

Sat Oct 11 12:36:14 2025 AUC 0.920124144668092

Sat Oct 11 12:37:04 2025 AUC 0.923874868795407

Sat Oct 11 12:39:11 2025 AUC 0.922274268469908

Sat Oct 11 12:40:26 2025 AUC 0.92576233511168

Sat Oct 11 12:42:52 2025 AUC 0.924306445824256

Sat Oct 11 12:44:10 2025 AUC 0.917302257967317

Sat Oct 11 12:44:40 2025 AUC 0.921402426038639

Sat Oct 11 12:48:56 2025 AUC 0.925953363230855

Sat Oct 11 12:49:38 2025 AUC 0.920447640924821

Sat Oct 11 12:53:46 2025 AUC 0.925676658446226

Sat Oct 11 12:57:28 2025 AUC 0.927310160

In [ ]:

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

[1] "num_iterations"          "learning_rate"          
 [3] "feature_fraction"        "num_leaves"             
 [5] "min_data_in_leaf"        "lambda_l1"              
 [7] "lambda_l2"               "bagging_fraction"       
 [9] "bagging_freq"            "min_gain_to_split"      
[11] "min_sum_hessian_in_leaf" "max_depth"              
[13] "max_bin"                 "y"                      
[15] "dob"                     "eol"                    
[17] "error.message"           "exec.time"              
[19] "ei"                      "error.model"            
[21] "train.time"              "prop.type"              
[23] "propose.time"            "se"                     
[25] "mean"

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]


PARAM$out$lgbm$y <- tb_bayesiana[1, y]


In [ ]:
write_yaml( PARAM, file="PARAM.yml")

In [ ]:
print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

   num_iterations learning_rate feature_fraction num_leaves min_data_in_leaf
            <int>         <num>            <num>      <int>            <int>
1:           2000     0.0210974        0.7768085        134               15
   lambda_l1 lambda_l2 bagging_fraction bagging_freq min_gain_to_split
       <num>     <num>            <num>        <int>             <num>
1:   1.28843 0.3073382        0.8910694            0        0.01095669
   min_sum_hessian_in_leaf max_depth max_bin
                     <num>     <int>   <int>
1:                1.724301        11     198
[1] 0.9377965


## 2.3  Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

In [ ]:
setwd("/content/buckets/b1/exp")
experimento <- paste0("exp", PARAM$experimento)
dir.create(experimento, showWarnings= FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la  optimización bayesiana

In [ ]:
# clase01
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

In [ ]:
dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

clase_ternaria,N
<fct>,<int>
CONTINUA,481491
BAJA+2,2808
BAJA+1,2492


In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

#### Final Training Hyperparameters

In [ ]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 99991

$max_depth
[1] 11

$min_gain_to_split
[1] 0.01095669

$min_sum_hessian_in_leaf
[1] 1.724301

$lambda_l1
[1] 1.28843

$lambda_l2
[1] 0.3073382

$max_bin
[1] 198

$bagging_fraction
[1] 0.8910694

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 2000

$learning_rate
[1] 0.0210974

$feature_fraction
[1] 0.7768085

$num_leaves
[1] 134

$min_data_in_leaf
[1] 15

$bagging_freq
[1] 0

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria y mucho menos cross validation.

In [ ]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

In [ ]:
# instalo y cargo la libreria  primes
if (!require("primes")) install.packages("primes")
require("primes")

In [ ]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(99991) # inicializo con mi primer semilla
# me quedo con por ejemplo 20 primos al azar
semillas_finales <- sample(primos, 10 )
print( semillas_finales )

 [1] 344417 905647 164371 908359 395287 271729 877771 102761 859423 246049


In [ ]:
# ALTERNATIVA
PARAM$future = c(202104)
dfuture <- dataset[foto_mes %in% PARAM$future]

# 1: Inicializar las listas necesarias para el loop
modelos_finales <- list()
predicciones_finales <- list()
resultados_ganancia_semilla <- list()

# Asegurar que PARAM$cortes está definido antes de los loops
PARAM$cortes <- seq(6000, 13000, by= 100)

# ----------------------------------------------------------------------
# LOOP PRINCIPAL: ENTRENAMIENTO Y EVALUACIÓN POR SEMILLA
# ----------------------------------------------------------------------
for (sem in semillas_finales) {
    param_normalizado$seed <- sem

    # ENTRENAR
    modelo <- lgb.train(
      data = dtrain_final,
      param = param_normalizado
    )

    # PREDECIR
    pred <- predict(
      modelo,
      data.matrix(dfuture[, campos_buenos, with=FALSE])
    )

    # Guardar la predicción y el modelo
    modelos_finales[[as.character(sem)]] <- modelo
    predicciones_finales[[as.character(sem)]] <- pred


    # EVALUAR GANANCIA INDIVIDUAL

    # Usar la variable 'pred' local o la que acabamos de guardar
    tb_pred_individual <- dfuture[, list(numero_de_cliente, foto_mes)]
    tb_pred_individual[, prob := pred ] # Usar la predicción local 'pred'

    # Ordenar por probabilidad descendente
    setorder(tb_pred_individual, -prob)

    drealidad <- realidad_inicializar( dfuture, PARAM)

    ganancias_semilla_actual <- data.table(
        semilla = sem,
        envios = integer(),
        ganancia_total = numeric(),
        ganancia_public = numeric(),
        ganancia_private = numeric()
    )

    # Evaluar la ganancia para todos los cortes
    for (envios in PARAM$cortes) {
        tb_pred_individual[, Predicted := 0L]
        tb_pred_individual[1:envios, Predicted := 1L]

        res <- realidad_evaluar( drealidad, tb_pred_individual)

        # Guardar los resultados
        ganancias_semilla_actual <- rbind(ganancias_semilla_actual,
            list(sem, envios, res$total, res$public, res$private)
        )
    }

    # Almacenar el data.table de ganancias para esta semilla
    resultados_ganancia_semilla[[as.character(sem)]] <- ganancias_semilla_actual
}

In [ ]:
require( "data.table" )

# 1. Extraer todas las predicciones de probabilidad de la lista
# La lista 'predicciones_finales' contiene un vector de probabilidades por semilla
lista_de_probs <- predicciones_finales

# 2. Convertir los vectores de probabilidad a un data.table
# Necesitamos una columna de ID (numero_de_cliente) para hacer el merge/promedio

# Crear el data.table base usando los identificadores de cliente del dfuture
dt_ensemble_202104 <- dfuture[, list(numero_de_cliente, foto_mes)]


# 3. Iterar sobre las predicciones y agregar cada una como una nueva columna
i <- 1
for (sem in names(lista_de_probs)) {
    # Crear un data.table temporal con la probabilidad y el ID
    dt_temp <- data.table(
        numero_de_cliente = dt_ensemble_202104$numero_de_cliente,
        # Aseguramos el nombre de columna dinámico, ej: 'prob_17', 'prob_2025'
        prob = lista_de_probs[[sem]]
    )

    # Renombrar la columna de probabilidad para evitar conflictos en el merge
    setnames(dt_temp, "prob", paste0("prob_", sem))

    # Mergear la nueva probabilidad al data.table principal
    dt_ensemble_202104 <- merge(
        dt_ensemble_202104,
        dt_temp,
        by = "numero_de_cliente"
    )
    i <- i + 1
}


# 4. Calcular la Probabilidad Promedio (Ensemble Final)
# Seleccionamos todas las columnas que empiezan con 'prob_'
cols_probabilidad <- grep("^prob_", names(dt_ensemble_202104), value = TRUE)

dt_ensemble_202104[, prob_ensemble := rowMeans(.SD), .SDcols = cols_probabilidad]

# 5. Generar la tabla de predicción final (solo ID y probabilidad)
prediccion_202104_ensamble <- dt_ensemble_202104[,
    list(numero_de_cliente, foto_mes, prob = prob_ensemble)
]

cat("Predicción ensemble para 202104 calculada con éxito.\n")

Predicción ensemble para 202104 calculada con éxito.


In [ ]:
# Definir el nombre del archivo de salida
archivo_prediccion <- paste0("prediccion_202104_ensemble_", PARAM$ID_MODELO, ".txt")

# Usar la función fwrite para guardar el data.table en formato .txt de forma eficiente
fwrite(prediccion_202104_ensamble,
       file = archivo_prediccion,
       sep = "\t",
       col.names = TRUE # Incluir los nombres de las columnas (numero_de_cliente, foto_mes, prob)
)

cat("Predicción ensemble 202104 guardada en:", archivo_prediccion, "\n")

Predicción ensemble 202104 guardada en: prediccion_202104_ensemble_.txt 


### Scoring

Kaggle Competition Submit

In [ ]:
PARAM$cortes <- seq(6000, 13000, by= 100)
PARAM$cortes

[1]  6000  6100  6200  6300  6400  6500  6600  6700  6800  6900  7000  7100
[13]  7200  7300  7400  7500  7600  7700  7800  7900  8000  8100  8200  8300
[25]  8400  8500  8600  8700  8800  8900  9000  9100  9200  9300  9400  9500
[37]  9600  9700  9800  9900 10000 10100 10200 10300 10400 10500 10600 10700
[49] 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900
[61] 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000

In [ ]:
# ----------------------------------------------------------------------
# EVALUACIÓN DEL ENSEMBLE (Se mantiene sin cambios importantes)
# ----------------------------------------------------------------------

# --- 1. Cálculo del Ensemble de Predicciones ---

# El promedio de predicciones (ensemble) se calcula sobre el total de semillas
prediccion_ensemble <- Reduce("+", predicciones_finales) / length(predicciones_finales)

# Se guarda el ensemble en la tabla de predicción final (tb_prediccion)
tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion_ensemble ]

# La realidad (drealidad) se inicializa una sola vez para la evaluación
drealidad <- realidad_inicializar( dfuture, PARAM)

# Ordenar por probabilidad descendente para el scoring
setorder(tb_prediccion, -prob)


# --- 2. Evaluación de la Ganancia del Ensemble ---

ganancias_ensemble <- data.table(
    envios = integer(),
    ganancia_total = numeric(),
    ganancia_public = numeric(),
    ganancia_private = numeric()
)

# Iterar sobre los cortes definidos para encontrar el óptimo del ensemble
for (envios in PARAM$cortes) {

    tb_prediccion[, Predicted := 0L]
    tb_prediccion[1:envios, Predicted := 1L]

    # Evaluar la ganancia
    res <- realidad_evaluar( drealidad, tb_prediccion)

    # Almacenar los resultados del ensemble
    ganancias_ensemble <- rbind(ganancias_ensemble,
        list(envios, res$total, res$public, res$private)
    )

    # Imprimir el resultado de cada corte del ensemble (opcional)
    options(scipen = 999)
    cat( "ENSEMBLE Envios=", envios, "\t",
        " TOTAL=", res$total,
        " Public=", res$public,
        " Private=", res$private,
        "\n",
        sep= ""
    )
}

# --- 3. Resultados Finales del Ensemble ---

# Encontrar el corte óptimo del ensemble (el que se usará para la predicción final)
corte_optimo_ensemble <- ganancias_ensemble[which.max(ganancia_total)]

print("\n--- Resultados Óptimos del Ensemble ---")
print(corte_optimo_ensemble)

ENSEMBLE Envios=6000	 TOTAL=335200000 Public=267600000 Private=364171429
ENSEMBLE Envios=6100	 TOTAL=336400000 Public=271266667 Private=364314286
ENSEMBLE Envios=6200	 TOTAL=340000000 Public=284733333 Private=363685714
ENSEMBLE Envios=6300	 TOTAL=343600000 Public=293266667 Private=365171429
ENSEMBLE Envios=6400	 TOTAL=344800000 Public=299000000 Private=364428571
ENSEMBLE Envios=6500	 TOTAL=345200000 Public=299533333 Private=364771429
ENSEMBLE Envios=6600	 TOTAL=345600000 Public=305133333 Private=362942857
ENSEMBLE Envios=6700	 TOTAL=345200000 Public=305400000 Private=362257143
ENSEMBLE Envios=6800	 TOTAL=347200000 Public=306266667 Private=364742857
ENSEMBLE Envios=6900	 TOTAL=347600000 Public=310066667 Private=363685714
ENSEMBLE Envios=7000	 TOTAL=346400000 Public=307866667 Private=362914286
ENSEMBLE Envios=7100	 TOTAL=347600000 Public=305333333 Private=365714286
ENSEMBLE Envios=7200	 TOTAL=346400000 Public=303333333 Private=364857143
ENSEMBLE Envios=7300	 TOTAL=349200000 Public=306933

In [ ]:
library(data.table)
library(ggplot2)
library(scales) # Para formatear los ejes

# 1. Consolidar las ganancias de las semillas en un solo data.table
# Ya deberías tener 'df_ganancias_consolidadas' del código anterior.
# Si no, ejecútalo:
df_ganancias_consolidadas <- rbindlist(resultados_ganancia_semilla)

# 2. Calcular la ganancia promedio de las semillas
df_ganancia_promedio <- df_ganancias_consolidadas[,
    list(
        ganancia_private = mean(ganancia_private)
    ),
    by = envios]
df_ganancia_promedio[, tipo := "Promedio de Semillas"]
df_ganancia_promedio[, semilla := "Promedio"] # Para la leyenda

# 3. Preparar los datos del Ensemble (el Ensemble es el que usarás para predecir)
df_ensemble_plot <- ganancias_ensemble[, list(envios, ganancia_private = ganancia_private)]
df_ensemble_plot[, tipo := "Ensemble (Predicción Final)"]
df_ensemble_plot[, semilla := "Ensemble"] # Para la leyenda

# 4. Consolidar todas las curvas en un solo data.table para ggplot2
df_plot_final <- rbind(
    df_ganancias_consolidadas[, .(envios, ganancia_private, tipo = "Semilla Individual", semilla = as.character(semilla))],
    df_ensemble_plot[, .(envios, ganancia_private, tipo, semilla)]
)

In [ ]:
# Exportar las ganancias de cada semilla (consolidado)
dir.create("ganancias", showWarnings = FALSE)

ruta_ganancias_por_semilla <- paste0("./ganancias/ganancia_semillas_", PARAM$experimento, "_", PARAM$semilla_primigenia, ".csv")

fwrite(df_ganancias_consolidadas,
       file = ruta_ganancias_por_semilla,
       sep = ",")

ruta_ganancias_ensemble <- paste0("./ganancias/ganancia_ensamble_", PARAM$experimento, ".csv")

fwrite(ganancias_ensemble,
       file = ruta_ganancias_ensemble,
       sep = ",")

cat("Archivos .csv guardados exitosamente en el directorio 'ganancias/'.\n")

Archivos .csv guardados exitosamente en el directorio 'ganancias/'.


In [ ]:
write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Oct 11 23:32:22 2025"

Finalmente usted deberá cargar el resultado de su corrida en la Google Sheet Colaborativa,  hoja **TareaHogar04**
<br> Siéntase libre de agregar las columnas que hagan falta a la planilla

PREDICCION FINAL

In [ ]:
# clase01
PARAM$train_final <- c(202101, 202102, 202103, 202104)
PARAM$future <- c(202106)
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]
dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

clase_ternaria,N
<fct>,<int>
CONTINUA,642824
BAJA+2,3938
BAJA+1,3447


In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

In [ ]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 99991

$max_depth
[1] 11

$min_gain_to_split
[1] 0.01095669

$min_sum_hessian_in_leaf
[1] 1.724301

$lambda_l1
[1] 1.28843

$lambda_l2
[1] 0.3073382

$max_bin
[1] 198

$bagging_fraction
[1] 0.8910694

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 2000

$learning_rate
[1] 0.0210974

$feature_fraction
[1] 0.7768085

$num_leaves
[1] 134

$min_data_in_leaf
[1] 15

$bagging_freq
[1] 0

In [ ]:
param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

In [ ]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(99991) # inicializo con mi primer semilla
# me quedo con por ejemplo 20 primos al azar
semillas_ensemble <- sample(primos, 10 )
print( semillas_ensemble )

 [1] 344417 905647 164371 908359 395287 271729 877771 102761 859423 246049


In [ ]:
dfuture <- dataset[foto_mes %in% PARAM$future]
matriz_prediccion_future <- data.matrix(dfuture[, campos_buenos, with= FALSE])
predicciones_individuales <- list()
importancias_individuales <- list()
cat("Iniciando entrenamiento y predicción para el Ensemble con", length(semillas_ensemble), "semillas...\n")

for (sem in semillas_ensemble) {
    param_normalizado$seed <- sem # Fijar la semilla para esta corrida

    cat(paste("Entrenando modelo con semilla:", sem, "\n"))

    # Entrenar LightGBM
    modelo <- lgb.train(
        data= dtrain_final,
        param= param_normalizado
    )

    # Aplicar el modelo a los datos futuros (202106)
    pred <- predict(modelo, matriz_prediccion_future)

    # Guardar la predicción
    predicciones_individuales[[as.character(sem)]] <- pred

    tb_importancia_semilla <- as.data.table(lgb.importance(modelo))
    importancias_individuales[[as.character(sem)]] <- tb_importancia_semilla
}

Iniciando entrenamiento y predicción para el Ensemble con 10 semillas...
Entrenando modelo con semilla: 344417 
Entrenando modelo con semilla: 905647 
Entrenando modelo con semilla: 164371 
Entrenando modelo con semilla: 908359 
Entrenando modelo con semilla: 395287 
Entrenando modelo con semilla: 271729 
Entrenando modelo con semilla: 877771 
Entrenando modelo con semilla: 102761 
Entrenando modelo con semilla: 859423 
Entrenando modelo con semilla: 246049 


In [ ]:
# 3.1. Promedio de predicciones (ENSEMBLE)
prediccion_ensemble <- Reduce("+", predicciones_individuales) / length(predicciones_individuales)

# 3.2. Crear tabla de predicción
tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion_ensemble ]

# 3.3. Grabar la probabilidad (Opcional, para debug)
fwrite(tb_prediccion,
    file= "prediccion_final_4923_ensemble.txt",
    sep= "\t"
)

In [ ]:
# 1. Consolidar todas las tablas de importancia individuales en una sola
df_importancias_consolidadas <- rbindlist(importancias_individuales, use.names = TRUE, fill = TRUE)

# 2. Calcular la Importancia Promedio por Feature
# Agrupamos por Feature y promediamos el valor de la importancia (Gain)
tb_importancia_ensemble <- df_importancias_consolidadas[,
    list(
        # Promediamos la métrica de importancia (Gain es la estándar)
        Gain_promedio = mean(Gain, na.rm = TRUE),
        # Opcional: Desviación estándar para medir la estabilidad de la importancia
        Gain_sd = sd(Gain, na.rm = TRUE)
    ),
    by = Feature]

# 3. Ordenar por la importancia promedio descendente
setorder(tb_importancia_ensemble, -Gain_promedio)


# 4. Guardar la importancia final del Ensemble
archivo_importancia_ensemble <- "impo_ensemble_4923.txt"

fwrite(tb_importancia_ensemble,
       file = archivo_importancia_ensemble,
       sep = "\t"
)

cat(paste("\nImportancia de variables del Ensemble guardada en:", archivo_importancia_ensemble, "\n"))


Importancia de variables del Ensemble guardada en: impo_ensemble_4923.txt 


In [ ]:
require( "data.table" )

# Rango de envíos a exportar (Tu rango deseado)
PARAM$cortes_kaggle <- seq(8500, 12000, by= 100)

# Crear la carpeta 'kaggle' si no existe
dir.create("kaggle", showWarnings = FALSE)

# 1. Ordenar el data.table de predicción una sola vez (por probabilidad)
setorder(tb_prediccion, -prob)

# 2. Loop para generar y exportar la submission para cada corte
cat("Iniciando la exportación de archivos Kaggle para", length(PARAM$cortes), "cortes...\n")
archivos_generados <- 0

# Creamos una copia de la tabla base para el loop
tb_export <- copy(tb_prediccion)

for (envios in PARAM$cortes_kaggle) {

    # 2a. Inicializar/Resetear la columna de predicción binaria
    tb_export[, Predicted := 0L]

    # 2b. Marcar como 1 (BAJA) solo a los N clientes con mayor probabilidad
    tb_export[1:envios, Predicted := 1L]

    # 2c. Definir el nombre del archivo
    # Usamos 'KA_C' para identificar que es un archivo de Corte
    archivo_kaggle <- paste0("./kaggle/4923_C", envios, ".csv")

    # 2d. Exportar la submission
    fwrite(tb_export[, .(numero_de_cliente, Predicted)],
           file = archivo_kaggle,
           sep = ",")

    archivos_generados <- archivos_generados + 1
}

# 3. Mensaje de Finalización
cat("\n--- TAREA FINALIZADA ---\n")
cat(paste("Se generaron", archivos_generados, "archivos de submission de Kaggle en la carpeta 'kaggle'.\n"))

Iniciando la exportación de archivos Kaggle para 71 cortes...

--- TAREA FINALIZADA ---
Se generaron 36 archivos de submission de Kaggle en la carpeta 'kaggle'.
